In [ ]:
# default_exp view_seg

In [ ]:
# export
import openslide
from pathlib import Path
import pickle
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from argparse import ArgumentParser
import PIL.Image as Image

In [ ]:
# export
SEG_DATA = Path('/data/DeepLearning/SCHWOB_Robin/Seg_outputs_ds')
IMG_DATA = Path('/data/DeepLearning/SCHWOB_Robin/Patches_normalized_299/Test')
GRADE = '1'
SCAN = 'CF_PACS05HE_15008-33373'
ITER = 3
SCAN_PATH = IMG_DATA/GRADE/SCAN
_SEGMENT_CSV = SEG_DATA/'Segmentation'/f'iter_{ITER}'/GRADE/f'{SCAN}.csv'
SEGMENT_CSV = SEG_DATA/'Segmentation'/GRADE/f'{SCAN}.csv'
_GRAPH_PATH = SEG_DATA/f'Tree/iter_{ITER}/most_significant_tree.p'
GRAPH_PATH = SEG_DATA/f'Tree/most_significant_tree.p'
_SEGMENT_STACK = SEG_DATA/f'Segmentation/iter_{ITER}/segment_stack.csv'
SEGMENT_STACK = SEG_DATA/f'Segmentation/segment_stack.csv'
OUTFOLDER = IMG_DATA/'Segmentation'

In [ ]:
# export
parser = ArgumentParser()
parser.add_argument('--file', '-f', help='notebook convenience')
parser.add_argument('--HistoryManager.hist_file', help='nbconvert convenience')
parser.add_argument('--seg-data', default=SEG_DATA, type=Path, help='Path to clustering output folder')
parser.add_argument('--img-data', default=IMG_DATA, type=Path, help='Path to image folder')
parser.add_argument('--iter', default=ITER, type=int, help='Iteration to load clustering info from')
parser.add_argument('--outfolder', default=OUTFOLDER, type=Path, help='Folder to store segments in')

_StoreAction(option_strings=['--outfolder'], dest='outfolder', nargs=None, const=None, default=PosixPath('/data/DeepLearning/SCHWOB_Robin/Patches_normalized_299/Test/Segmentation'), type=<class 'pathlib.Path'>, choices=None, help='Folder to store segments in', metavar=None)

In [ ]:
# export
args = parser.parse_args()

In [ ]:
# export
segment_stack = args.seg_data/f'Segmentation/iter_{args.iter}/segment_stack.csv'
df = pd.read_csv(segment_stack)

In [ ]:
# export
graph_path = args.seg_data/f'Tree/iter_{args.iter}/most_significant_tree.p'
with graph_path.open('rb') as f:
    tree = pickle.load(f)

FileNotFoundError: [Errno 2] No such file or directory: '/data/DeepLearning/SCHWOB_Robin/Seg_outputs_ds/Tree/most_significant_tree.p'

In [ ]:
# export
centroids = np.stack([tree.descriptor[k] for k in (13357, 13149, 13134, 21613)])

In [ ]:
# export
colors = np.array([[155, 232, 157],
                   [237, 7, 7],
                   [245, 17, 199],
                   [92, 87, 91]], dtype=np.uint8)

In [ ]:
# export
for grade_fold in args.img_data.iterdir():
    grade = grade_fold.name
    for scan_fold in grade_fold.iterdir():
        scan_name = scan_fold.name
        print(scan_name)
        sub_df = df.loc[df['Slidename'] == scan_name]
        features = sub_df.values[:, 3:].astype(np.float32)
        result = np.argmin(((features[None] - centroids[:, None])**2).sum(-1), axis=0)
        
        segment_csv = args.seg_data/'Segmentation'/f'iter_{args.iter}'/grade/f'{scan_name}.csv'
        scan_df = pd.read_csv(segment_csv, usecols=(0, 1, 2))
        segment = np.zeros((scan_df['YPosition'].max() // 299 + 1, scan_df['XPosition'].max() // 299 + 1, 3), dtype=np.uint8)
        for _, row in scan_df.iterrows():
            segment[row.YPosition//299, row.XPosition//299] = colors[result[row.Label]]
        outfile = args.outfolder/grade/f'{scan_name}.npy'
        if not outfile.parent.is_dir():
            outfile.parent.mkdir(parents=True)
        np.save(outfile, segment)
        segment = Image.fromarray(segment)
        segment.save(outfile.with_suffix('.png'))

CF_PACS04rescan_08P0124
CF_PACS04rescan_06P1204


In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script(fname='_view_seg.ipynb')

Converted _view_seg.ipynb.
